# Perturbations in SE(3) using Lie-Group Theory Using `PyLie`


Install libraries

In [ ]:
!pip -q install https://github.com/tussedrotten/pylie/archive/master.zip
!pip -q install https://github.com/tussedrotten/visgeom/archive/master.zip

## Visualizing Perturbations on $SE(3)$

This visualization shows how **small perturbations (twists) in the Lie algebra**
$\mathfrak{se}(3)$ act on a rigid-body pose in the Lie group $SE(3)$.  
It illustrates a fundamental concept in Lie-group theory: **a perturbation is not
just added to a pose; it must be applied through a group action**, and the result
depends on *where* the perturbation is applied.

---

### Fixed Reference Pose

A fixed pose
$
T_{w a} \in SE(3)
$
is chosen as an **anchor frame**. It represents a rigid-body pose (i.e., orientation and
position) expressed in the world frame. All perturbations are visualized relative to
this anchor.

---

### The Perturbation (Twist)

A perturbation is represented by a **twist vector**
$
\boldsymbol{\xi} \in \mathfrak{se}(3),
$
where only one component $\xi_1$ is varied. This means that the visualization explores
a **single basis direction of the tangent space**.

This infinitesimal motion $\boldsymbol{\xi}$ is converted  into a finite rigid-body transformation by the *exponential map*
$
\exp(\boldsymbol{\xi}) \in SE(3).
$


---

## Perturbation Modes

The example in this notebook shows three different ways of
applying the same tangent-space perturbation.

---

### 1. Pure Exponential (`MODE = "exp"`)

$
T(\boldsymbol{\xi}) = \exp(\boldsymbol{\xi})
$

- The perturbation is applied at the **identity** of the group.
- This shows the rigid motion represented by the twist itself.
- No reference pose is involved.

This answers the question:  
*“What rigid-body motion does this twist represent on its own?”*


### 2. Right Perturbation (`MODE = "right"`) — Body-Frame Error

$
T(\boldsymbol{\xi}) = T_{w a}\,\exp(\boldsymbol{\xi})
$

- The perturbation is applied on the **right** of the pose.
- The twist is expressed in the **body (local) frame**.
- Perturbation directions rotate with the object.

This corresponds to:
- Right-invariant errors
- Body-frame error models
- Commonly used in error-state Kalman filters (ESKF)

Interpretation:  
*“How does the pose change if the object itself moves slightly?”*


### 3. Left Perturbation (`MODE = "left"`) — World-Frame Error

$
T(\boldsymbol{\xi}) = \exp(\boldsymbol{\xi})\,T_{w a}
$

- The perturbation is applied on the **left** of the pose.
- The twist is expressed in the **world frame**.
- Perturbation directions remain fixed in global coordinates.

This corresponds to:
- Left-invariant errors
- World-frame perturbations
- Residual formulations common in pose-graph optimization (e.g., SLAM).

Interpretation:  
*“How does the pose change if the world frame perturbs it?”*

---

## Geodesic Trajectory on the Manifold

For each mode, a faint trajectory is drawn:
$
T(\alpha) = T_{\text{base}_1}\,\exp(\alpha\,\boldsymbol{\xi})\,T_{\text{base}_2},
\qquad \alpha \in [0,1].
$

This curve represents:
- Linear motion in the tangent space $\mathfrak{se}(3)$
- A **geodesic** (shortest path) on the manifold $SE(3)$
- Motion under a **constant twist**

Scaling $\alpha$ moves linearly in the Lie algebra, while the exponential map
retracts the motion onto the curved manifold.

---

## Summary

- Perturbations on $SE(3)$ are **not vectors**; they are elements of the tangent space.
- Applying the same perturbation on the left or on the right produces **different
  motions**.
- Left and right perturbations have different physical meanings and lead to different
  Jacobians.
- This distinction is fundamental in state estimation, uncertainty modeling, and
  optimization on Lie groups.

---

In [ ]:
#@title Visualize types of perturbations

# ============================================================
# Plotly notebook adaptation of vis_perturbations (single cell)
# ============================================================

import numpy as np
import plotly.graph_objects as go
from pylie import SO3, SE3

# ------------------------------------------------------------
# Helper: draw a coordinate frame (Plotly version of vg.plot_pose)
# ------------------------------------------------------------
def plotly_frame(data, R, t, scale=1.0, opacity=1.0):
    t = t.flatten()
    colors = ["red", "green", "blue"]
    for i in range(3):
        axis = t + scale * R[:, i]
        data.append(
            go.Scatter3d(
                x=[t[0], axis[0]],
                y=[t[1], axis[1]],
                z=[t[2], axis[2]],
                mode="lines",
                line=dict(color=colors[i], width=6),
                opacity=opacity,
                showlegend=False,
            )
        )

# ------------------------------------------------------------
# Fixed frame T_w_a (same as original code)
# ------------------------------------------------------------
T_w_a = SE3((
    SO3.from_roll_pitch_yaw(5*np.pi/4, 0, np.pi/2),
    np.array([[2, 2, 2]]).T
))

#@markdown Choose perturbation mode:
MODE = "exp"  #@param ["exp", "right", "left"]


# ------------------------------------------------------------
# Select which twist component ξ_i to perturb
# ------------------------------------------------------------
XI_INDEX = 5  #@param {type:"slider", min:0, max:5, step:1}

# Labels
XI_LABELS = {
    0: "ρₓ  (translation x)",
    1: "ρᵧ  (translation y)",
    2: "ρ_z  (translation z)",
    3: "φₓ  (rotation x)",
    4: "φᵧ  (rotation y)",
    5: "φ_z  (rotation z)",
}

print(f"Perturbing twist component: ξ_{XI_INDEX} = {XI_LABELS[XI_INDEX]}")



# ------------------------------------------------------------
# Precompute frames (Plotly animation model)
# We vary ONE component of xi (xi_1), like a slider
# ------------------------------------------------------------
vals = np.linspace(-3.0, 3.0, 31)
frames = []

for v in vals:
    xi = np.zeros((6, 1))
    xi[XI_INDEX, 0] = v   # perturb ξ₁

    data = []

    # World frame F_w
    plotly_frame(data, *SE3().to_tuple(), scale=1.0)

    # Anchor frame F_a (only for right / left)
    if MODE in ["right", "left"]:
        plotly_frame(data, *T_w_a.to_tuple(), scale=1.0)

    # Compute perturbed pose
    if MODE == "exp":
        T = SE3.Exp(xi)
        T_base_1 = SE3()
        T_base_2 = SE3()

    elif MODE == "right":
        T = T_w_a @ SE3.Exp(xi)
        T_base_1 = T_w_a
        T_base_2 = SE3()

    else:  # left
        T = SE3.Exp(xi) @ T_w_a
        T_base_1 = SE3()
        T_base_2 = T_w_a

    # Plot perturbed pose
    plotly_frame(data, *T.to_tuple(), scale=1.0)

    # Manifold trajectory (same logic as draw_interpolated)
    for alpha in np.linspace(0, 1, 20):
        Ti = T_base_1 @ SE3.Exp(alpha * xi) @ T_base_2
        plotly_frame(data, *Ti.to_tuple(), scale=0.6, opacity=0.15)

    frames.append(
        go.Frame(
            data=data,
            name=f"{v:.2f}"
        )
    )

# ------------------------------------------------------------
# Base figure
# ------------------------------------------------------------
fig = go.Figure(
    data=frames[len(frames)//2].data,
    frames=frames
)

# ------------------------------------------------------------
# Slider (animation-based, notebook-safe)
# ------------------------------------------------------------
fig.update_layout(
    sliders=[{
        "steps": [
            {
                "args": [[f"{v:.2f}"], {"frame": {"duration": 0}, "mode": "immediate"}],
                "label": f"{v:.2f}",
                "method": "animate",
            }
            for v in vals
        ],
        "currentvalue": {"prefix": "ξ₁ = "},
        "pad": {"t": 30},
    }],
    scene=dict(
        aspectmode="data",
        bgcolor="white",
        xaxis=dict(
            range=[-4, 4],
            showbackground=False,
            showgrid=False,
            zeroline=False,
            showticklabels=False,
            ticks="",
            title="",
        ),
        yaxis=dict(
            range=[-4, 4],
            showbackground=False,
            showgrid=False,
            zeroline=False,
            showticklabels=False,
            ticks="",
            title="",
        ),
        zaxis=dict(
            range=[-4, 4],
            showbackground=False,
            showgrid=False,
            zeroline=False,
            showticklabels=False,
            ticks="",
            title="",
        ),
    ),
    paper_bgcolor="white",
    plot_bgcolor="white",
    title=f"SE(3) Perturbations on the Manifold (mode = {MODE})",
    margin=dict(l=0, r=0, t=50, b=0),
)

fig
